In [1]:
import requests
from owslib.ogcapi.features import Features
from pystac_client import Client

import obstore as obs
from obstore.store import from_url, AzureStore
from obstore.auth.planetary_computer import PlanetaryComputerCredentialProvider
import planetary_computer

import shapely.geometry
import mercantile

import pyarrow.parquet as pq

import deltalake
import adlfs

from lonboard import viz
import pandas as pd
import geopandas as gpd
import jupytergis

import rasterio as rs
from rasterio.plot import show

In [75]:
url = "https://openveda.cloud/api/features/"

In [76]:
w = Features(url)
w.feature_collections()

['public.eis_fire_lf_perimeter_archive',
 'public.eis_fire_lf_newfirepix_archive',
 'public.eis_fire_lf_fireline_archive',
 'public.eis_fire_lf_fireline_nrt',
 'public.eis_fire_snapshot_fireline_nrt',
 'public.eis_fire_snapshot_newfirepix_nrt',
 'public.eis_fire_lf_newfirepix_nrt',
 'public.eis_fire_lf_perimeter_nrt',
 'public.eis_fire_snapshot_perimeter_nrt',
 'pg_temp.eis_fire_lf_perimeter_nrt_latest',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

In [77]:
perim = w.collection("public.eis_fire_snapshot_perimeter_nrt")

In [78]:
perim_q = w.collection_queryables("public.eis_fire_snapshot_perimeter_nrt")
perim_q["properties"]

{'geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'duration': {'name': 'duration', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'geom_counts': {'name': 'geom_counts', 'type': 'string'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 'low_confidence_grouping': {'name': 'low_confidence_grouping',
  'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'primarykey': {'name': 'primarykey', 'type': 'string'},
 'region': {'name': 'region', 'type': 'string'},
 't': {'name': 't', 'type': 'string'}}

In [2]:
new_mexico_polygon = shapely.geometry.box("-110", "31", "-103", "38") 
new_mexico = ["-110", "31", "-103", "38"]

In [80]:
perim_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    bbox=new_mexico,
    limit=1000
)

In [81]:
perimeters = gpd.GeoDataFrame.from_features(perim_results["features"])
perimeters

,geometry,duration,farea,fireid,flinelen,fperim,geom_counts,isactive,low_confidence_grouping,meanfrp,n_newpixels,n_pixels,pixden,primarykey,region,t
0,"MULTIPOLYGON (((-103.89592 31.8102, -103.8959 ...",171.0,1.417770,199,0.000000,8.126009,None,1,0,1.020,2,97,68.417300,CONUS|199|2025-06-21T00:00:00,CONUS,2025-06-21T00:00:00
1,"MULTIPOLYGON (((-103.65612 31.26333, -103.6560...",74.0,1.400332,33485,0.000000,5.963962,None,0,0,0.450,1,106,75.696326,CONUS|33485|2025-06-12T00:00:00,CONUS,2025-06-12T00:00:00
2,"MULTIPOLYGON (((-103.70191 31.98301, -103.7018...",66.0,12.149895,35957,1.838496,35.117333,None,0,0,0.635,2,64,5.267535,CONUS|35957|2025-06-12T00:00:00,CONUS,2025-06-12T00:00:00
3,"POLYGON ((-103.7252 31.03568, -103.7252 31.035...",59.0,0.528838,37645,2.642837,2.642837,None,0,0,0.950,1,15,28.364093,CONUS|37645|2025-06-07T00:00:00,CONUS,2025-06-07T00:00:00
4,"POLYGON ((-106.5868 31.96361, -106.58688 31.96...",70.0,0.530218,39065,0.000000,2.642315,None,1,0,2.250,1,28,52.808441,CONUS|39065|2025-06-20T00:00:00,CONUS,2025-06-20T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,"POLYGON ((-106.49197 31.60161, -106.49226 31.5...",0.0,0.314045,59329,2.259740,2.259740,None,1,0,2.070,3,3,9.552757,CONUS|59329|2025-06-22T00:00:00,CONUS,2025-06-22T00:00:00
89,"POLYGON ((-103.65441 32.42808, -103.65442 32.4...",0.0,0.141000,59333,1.177624,1.177624,None,1,0,0.640,1,1,7.092199,CONUS|59333|2025-06-22T00:00:00,CONUS,2025-06-22T00:00:00
90,"POLYGON ((-106.70971 34.82156, -106.70971 34.8...",0.0,0.141000,59476,1.177624,1.177624,None,1,0,0.400,1,1,7.092199,CONUS|59476|2025-06-23T00:00:00,CONUS,2025-06-23T00:00:00
91,"MULTIPOLYGON (((-106.61915 35.98528, -106.6191...",0.0,0.220539,59623,2.355248,2.355248,None,1,0,8.570,2,2,9.068709,CONUS|59623|2025-06-23T12:00:00,CONUS,2025-06-23T12:00:00


In [82]:
viz(perimeters)

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Buildings

In [83]:
mpc_url = "https://planetarycomputer.microsoft.com/api/stac/v1/"
mpc = Client.open(mpc_url)
mpc.title

'Microsoft Planetary Computer STAC API'

In [84]:
mpc_search = mpc.collection_search(
    q="building"
)

print(f"{mpc_search.matched()} collections found")

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


2 collections found


In [85]:
for collection in mpc_search.collections():
    print(collection.id)

3dep-lidar-classification
ms-buildings


In [86]:
buildings_cat = mpc.get_collection("ms-buildings")
buildings_cat.title

'Microsoft Building Footprints'

In [87]:
building_regions_search = mpc.search(
    collections=["ms-buildings"],
    bbox=new_mexico
).item_collection_as_dict()

In [88]:
building_regions = gpd.GeoDataFrame.from_features(building_regions_search["features"])
building_regions

,geometry,title,datetime,description,table:columns,table:row_count,msbuildings:region,end_datetime,start_datetime
0,"POLYGON ((-66.95046 18.92146, -66.95046 71.360...",Building footprints,2022-07-06T00:00:00Z,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",129554779,United States,NaN,NaN
1,"POLYGON ((-66.95046 18.92146, -66.95046 71.360...",Building footprints,2022-06-14T00:00:00Z,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",129557523,United States,NaN,NaN
2,"POLYGON ((-86.71319 14.53409, -86.71319 32.716...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",23769510,Mexico,2021-12-18T00:00:00+00:00,2017-06-10T00:00:00+00:00
3,"POLYGON ((-86.71319 14.53409, -86.71319 32.716...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",23769510,Mexico,2021-12-18T00:00:00+00:00,2017-06-10T00:00:00+00:00
4,"POLYGON ((-61.00254 7.39233, -61.00254 32.4727...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",3234,North America,2021-06-08T00:00:00+00:00,2014-06-19T00:00:00+00:00
5,"POLYGON ((-61.00254 7.39233, -61.00254 32.4727...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",3234,North America,2021-06-08T00:00:00+00:00,2014-06-19T00:00:00+00:00


In [139]:
search = mpc.search(
    collections=["ms-buildings"],
    intersects=new_mexico_polygon,
    query={
    # "msbuildings:region": {"eq": "United States"},
    # "msbuildings:processing-date": {"eq": "2023-04-25"}
    },
)

ic = search.item_collection()

In [140]:
list(ic)

[<Item id=United States_2022-07-06>,
 <Item id=United States_2022-06-14>,
 <Item id=Mexico_2022-07-06>,
 <Item id=Mexico_2022-06-14>,
 <Item id=North America_2022-07-06>,
 <Item id=North America_2022-06-14>]

In [152]:
ic[0]

<Item id=United States_2022-07-06>

In [133]:
parquet_file_info = ic[0].assets["data"]

In [134]:
parquet_file_info

<Asset href=abfs://footprints/global/2022-07-06/ml-buildings.parquet/RegionName=United States>

In [135]:
authenticate_request = requests.get(url=f"https://planetarycomputer.microsoft.com/api/sas/v1/token/{buildings_cat.id}")
token = authenticate_request.json()["token"]
storage_opts = parquet_file_info.extra_fields["table:storage_options"]

In [141]:
store = AzureStore(credential_provider=PlanetaryComputerCredentialProvider(parquet_file_info.href, account_name=storage_opts["account_name"]))

In [154]:
files = obs.list(store, return_arrow=True)

In [3]:
quadkeys = [
    int(mercantile.quadkey(tile))
    for tile in mercantile.tiles(*new_mexico_polygon.bounds, zooms=9)
]
quadkeys

[23100213,
 23100231,
 23100233,
 23102011,
 23102013,
 23102031,
 23102033,
 23102211,
 23102213,
 23102231,
 23102233,
 23120011,
 23120013,
 23100302,
 23100320,
 23100322,
 23102100,
 23102102,
 23102120,
 23102122,
 23102300,
 23102302,
 23102320,
 23102322,
 23120100,
 23120102,
 23100303,
 23100321,
 23100323,
 23102101,
 23102103,
 23102121,
 23102123,
 23102301,
 23102303,
 23102321,
 23102323,
 23120101,
 23120103,
 23100312,
 23100330,
 23100332,
 23102110,
 23102112,
 23102130,
 23102132,
 23102310,
 23102312,
 23102330,
 23102332,
 23120110,
 23120112,
 23100313,
 23100331,
 23100333,
 23102111,
 23102113,
 23102131,
 23102133,
 23102311,
 23102313,
 23102331,
 23102333,
 23120111,
 23120113,
 23101202,
 23101220,
 23101222,
 23103000,
 23103002,
 23103020,
 23103022,
 23103200,
 23103202,
 23103220,
 23103222,
 23121000,
 23121002,
 23101203,
 23101221,
 23101223,
 23103001,
 23103003,
 23103021,
 23103023,
 23103201,
 23103203,
 23103221,
 23103223,
 23121001,
 23121003,

In [97]:
parquet_urls = [f for f in files if any(f"quadkey={qk}" in f for qk in quadkeys)]
parquet_urls

[]

In [130]:
path = "part-00821-2abf88bf-ebfb-42fa-9843-5951065b6382.c000.snappy.parquet"

parquet_test = store.get(path).bytes()

with open("parquet_test.parquet", "wb") as f:
    f.write(parquet_test)

In [131]:
test_gpd = gpd.read_parquet("parquet_test.parquet")
test_gpd

,geometry
0,"POLYGON ((-100.00058 48.37437, -100.00058 48.3..."
1,"POLYGON ((-101.80731 33.56686, -101.80724 33.5..."
2,"POLYGON ((-102.38294 31.86974, -102.38291 31.8..."
3,"POLYGON ((-104.80376 39.35458, -104.80388 39.3..."
4,"POLYGON ((-100.4272 48.61329, -100.4272 48.613..."
...,...
649637,"POLYGON ((-93.33224 44.79579, -93.33223 44.795..."
649638,"POLYGON ((-95.438 29.92417, -95.43799 29.92424..."
649639,"POLYGON ((-96.18249 41.21011, -96.18245 41.210..."
649640,"POLYGON ((-97.39348 28.67374, -97.39361 28.673..."


In [132]:
viz(test_gpd)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

### Try deltalake

In [4]:
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

In [5]:
asset = collection.assets["delta"]

In [6]:
storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}

In [7]:
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

In [9]:
file_uris = table.file_uris([("RegionName", "=", "UnitedStates"),("quadkey", "in", quadkeys)])
file_uris

['az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102312/part-00005-b32140c2-71a7-4634-ad9b-e468f8e82e70.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102330/part-00005-c55f6fcd-acbc-4139-b202-de34f44a596c.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102320/part-00017-1fd60be6-eb2a-4146-ba86-32ce6dfb16b6.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103203/part-00017-6557efe5-1d9d-43f0-8667-8e9cbd92878f.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103213/part-00017-b344dc85-9f97-471d-ba37-2df8fa1a652f.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102113/part-00021-30322349-e3c4-4eaf-96ea-858e07a1bae3.c000.snappy.parquet'

In [14]:
gdf = pd.concat(
    [
        gpd.read_parquet(file_uri, storage_options=storage_options)
        for file_uri in file_uris
    ]
)

In [15]:
gdf

,geometry,meanHeight,RegionName,quadkey
0,"POLYGON ((-108.20774 33.19588, -108.20778 33.1...",2.908320,UnitedStates,23102312
1,"POLYGON ((-107.67342 33.33829, -107.67356 33.3...",-1.000000,UnitedStates,23102312
2,"POLYGON ((-107.59869 33.27274, -107.59853 33.2...",-1.000000,UnitedStates,23102312
3,"POLYGON ((-107.64454 33.34756, -107.64446 33.3...",-1.000000,UnitedStates,23102312
4,"POLYGON ((-107.64928 33.34947, -107.6493 33.34...",-1.000000,UnitedStates,23102312
...,...,...,...,...
89157,"POLYGON ((-106.08491 35.97431, -106.08491 35.9...",3.250824,UnitedStates,23103003
89158,"POLYGON ((-106.06174 35.62731, -106.06167 35.6...",3.672221,UnitedStates,23103003
89159,"POLYGON ((-106.07137 35.97642, -106.07133 35.9...",3.831281,UnitedStates,23103003
89160,"POLYGON ((-106.04049 35.682, -106.04044 35.681...",-1.000000,UnitedStates,23103003


## Population

In [101]:
pop_url = "https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km_UNadj/2020/USA/usa_pd_2020_1km_UNadj.tif" # From https://data.humdata.org/dataset/worldpop-population-density-for-united-states-of-america

In [102]:
# population = rs.open(pop_url)

---

Trying to do it in a fully STAC way

In [103]:
stac_url = "https://openveda.cloud/api/stac"

In [104]:
veda_catalog = Client.open(stac_url)
veda_catalog.title

'VEDA (Visualization, Exploration, and Data Analysis) STAC API'

In [105]:
snapshot_perimeter = veda_catalog.collection_search(
    q="fire"
)

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


In [106]:
snapshot_perimeter.matched()

19

In [107]:
for collection in snapshot_perimeter.collections():
    print(collection.id)

barc-thomasfire
caldor-fire-behavior
disalexi-etsuppression
caldor-fire-burn-severity
eis_fire_perimeter
campfire-albedo-wsa-diff
frp-max-thomasfire
campfire-lst-day-diff
campfire-lst-night-diff
campfire-nlcd
hls-bais2-v2
landsat-nighttime-thermal
campfire-ndvi-diff
lis-tvegsuppression
lis-etsuppression
la-fires-hrrr-wind
la-fires-planet
la-fires-frp
la-fires-maxar
